In [1]:
!docker login -u mariembouhadda -p sS58169362

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/jupyter/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [41]:
%%writefile Dockerfile
FROM python:3.7

#RUN git clone https://github.com/meriembouhadda/hello_world.git

COPY hello_world_launcher.py .
COPY hello_world.py . 
COPY requirements.txt .

RUN pip install -r requirements.txt
#ARG project_id='kheopsys-data-lab'
#ARG bucket='ml-pipeline-309409_bucket'
#using bash we run the deploy script we just copied in here
ENTRYPOINT ["python","hello_world_launcher.py"]
#,"--project","ml-pipeline-309409","--bucket","ml-pipeline-309409_bucket"]

Overwriting Dockerfile


In [42]:
!docker build -t mariembouhadda/hello-world-test .

Sending build context to Docker daemon  462.3kB
Step 1/6 : FROM python:3.7
 ---> d475516974f2
Step 2/6 : COPY hello_world_launcher.py .
 ---> 7170cc523a05
Step 3/6 : COPY hello_world.py .
 ---> 4f3f2829a942
Step 4/6 : COPY requirements.txt .
 ---> 076253e1ae58
Step 5/6 : RUN pip install -r requirements.txt
 ---> Running in 5b0218f707db
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.2/301.2 KB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 KB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.6/636.6 KB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 KB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 KB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 KB 8.8 MB/s eta 0:00

In [43]:
!docker push mariembouhadda/hello-world-test

Using default tag: latest
The push refers to repository [docker.io/mariembouhadda/hello-world-test]

d4194825: Preparing 
92ca174a: Preparing 
7fefe95d: Preparing 
1d8af70c: Preparing 
51a0e3c7: Preparing 
09d52b7b: Preparing 
3c789e6d: Preparing 
9a7a4e4a: Preparing 
9a205adc: Preparing 
98da17f2: Preparing 
69727d80: Preparing 
22fdcc61: Preparing 
4194825: Pushed   123.1MB/118.6MBdda/hello-world [12Alatest: digest: sha256:10720ac4229cbdd11509f2fe03c27357b2c43a6f1ea7702a21c675ed991bc5bc size: 3052


In [4]:
import kfp
import kfp.components as comp

In [20]:
@kfp.dsl.component
def hello_world():
    # Defining component configuration
    hello_component = kfp.dsl.ContainerOp(
        name='hello-world',
        image='docker.io/mariembouhadda/hello-world:latest',
        command=['python', 'hello_world.py'],
        )
    return hello_component

In [21]:
@kfp.dsl.pipeline(
  name="hello world script",
  description="hello world script"
)
def hi():
    hello = hello_world()
    hello.execution_options.caching_strategy.max_cache_staleness = "P0D"


In [22]:
kfp.compiler.Compiler().compile(hi, 'hello_world.zip')

In [23]:
client = kfp.Client(host='https://21306363b1ebd301-dot-us-central1.pipelines.googleusercontent.com')

In [24]:
EXPERIMENT_NAME = 'hello world test'
experiment = client.create_experiment(name=EXPERIMENT_NAME)

In [25]:
run = client.run_pipeline(experiment.id, 'hello world test', 'hello_world.zip')

In [12]:
#!pip install db-dtypes

In [13]:
#import db_dtypes